In [11]:
# Imports
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_leaflet as dl
import base64

# Import CRUD module to interact with MongoDB
from AnimalShelter import AnimalShelter

# MongoDB credentials
username = "aacuser"
password = "Frontporch1"

# Instantiate AnimalShelter class for DB operations
shelter = AnimalShelter(username, password)

# Rescue type filters with breeds, sex, and age range info
rescue_filters = {
    'water': {
        "breed": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
        "sex": "Intact Female",
        "age_min": 26,
        "age_max": 156
    },
    'mountain': {
        "breed": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
        "sex": "Intact Male",
        "age_min": 26,
        "age_max": 156
    },
    'disaster': {
        "breed": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"],
        "sex": "Intact Male",
        "age_min": 20,
        "age_max": 300
    }
}

# Encode logo image to base64 for embedding
image_filename = 'SalvareLogo.png'
encoded_logo = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Initialize Dash app
app = JupyterDash(__name__)

# Layout includes logo with link, creator text, dropdown filter, table, charts
app.layout = html.Div([
    # Header with logo linking to SNHU and my name as the creator
    html.Div([
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_logo), style={'height': '80px'}),
            href='https://www.snhu.edu',
            target='_blank',
            title='Grazioso Salvare Home'
        ),
        html.Div("Created by Steve Arevalo", style={
            'fontSize': '18px', 'fontWeight': 'bold',
            'paddingLeft': '20px', 'alignSelf': 'center'
        })
    ], style={'display': 'flex', 'alignItems': 'center', 'padding': '10px 0'}),

    # Title of the dashboard
    html.H1("Austin Animal Center Outcomes Dashboard"),

    # Dropdown to select rescue type or reset filter; no default selected (None)
    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.Dropdown(
            id='rescue-filter',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset (Show All Animals)', 'value': 'reset'}
            ],
            value=None,          # No selection at start
            clearable=True,      # Allow clearing selection
            style={'width': '50%'}
        )
    ], style={'padding': '20px 0'}),

    # Data table to show filtered animals; paging, filtering, sorting enabled
    dash_table.DataTable(
        id='animal-table',
        columns=[],
        data=[],
        page_size=10,
        filter_action='native',
        sort_action='native',
        row_selectable='single',
        style_table={'overflowX': 'auto'}
    ),

    html.Hr(),

    # Container with pie chart and map side-by-side
    html.Div(style={'display': 'flex', 'justifyContent': 'space-between'}, children=[
        html.Div(style={'width': '100%'}, children=[
            dcc.Graph(id='breed-pie-chart')  # Pie chart for breed distribution
        ]),
        html.Div(style={'width': '48%'}, children=[
            html.Div(id='map-container')      # Map showing selected animal location
        ])
    ])
])

# Callback to update table data, columns, and pie chart based on dropdown filter
@app.callback(
    [Output('animal-table', 'data'),
     Output('animal-table', 'columns'),
     Output('breed-pie-chart', 'figure')],
    [Input('rescue-filter', 'value')]
)
def update_data_table_and_pie(selected_filter):
    # If nothing selected or reset, query all animals (no filters)
    if selected_filter is None or selected_filter == 'reset':
        query = {}  # No filtering, returns all records
    else:
        # Filter by rescue type criteria plus ensure only dogs are returned
        f = rescue_filters[selected_filter]
        query = {
            "breed": {"$in": f["breed"]},
            "sex_upon_outcome": f["sex"],
            "age_upon_outcome_in_weeks": {"$gte": f["age_min"], "$lte": f["age_max"]},
            "animal_type": "Dog"
        }

    # Retrieve records from MongoDB using CRUD read method
    records = shelter.read(query)

    # Remove internal MongoDB _id from each record to avoid Dash errors
    for r in records:
        r.pop('_id', None)

    # Prepare columns and dataframe for Dash DataTable and chart
    if records:
        columns = [{"name": k, "id": k} for k in records[0].keys()]
        df = pd.DataFrame(records)
    else:
        columns = []
        df = pd.DataFrame()

    # Create breed distribution pie chart or empty placeholder if no data
    if not df.empty and 'breed' in df.columns:
        fig = px.pie(df, names='breed', title='Breed Distribution')
    else:
        fig = px.pie(title="No Data Available")

    # Return table data, columns, and pie chart figure to update dashboard
    return records, columns, fig

# Callback to update map based on selected row in data table
@app.callback(
    Output('map-container', 'children'),
    [Input('animal-table', 'derived_virtual_data'),
     Input('animal-table', 'derived_virtual_selected_rows')]
)
def update_map(rows, selected_rows):
    # Handle no data condition
    if not rows or len(rows) == 0:
        return html.P("No data available for map.")

    # Convert current table data to DataFrame for easy indexing
    dff = pd.DataFrame(rows)

    # Default selected row to first if none selected by user
    if selected_rows is None or len(selected_rows) == 0:
        row_idx = 0
    else:
        row_idx = selected_rows[0]

    # Ensure location columns exist in data
    if 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return html.P("No location data available.")

    lat = dff.iloc[row_idx]['location_lat']
    lon = dff.iloc[row_idx]['location_long']

    # Check for missing or invalid coordinates
    if pd.isna(lat) or pd.isna(lon):
        return html.P("Selected animal has no valid location data.")

    # Build and return map centered on selected animal location with marker & popup
    return dl.Map(
        style={'width': '100%', 'height': '500px'},
        center=[lat, lon],
        zoom=12,
        children=[
            dl.TileLayer(),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row_idx].get('breed', 'Unknown Breed')),
                dl.Popup([
                    html.H4("Animal Details"),
                    html.P(f"Name: {dff.iloc[row_idx].get('name', 'N/A')}"),
                    html.P(f"Breed: {dff.iloc[row_idx].get('breed', 'N/A')}"),
                    html.P(f"Age (weeks): {dff.iloc[row_idx].get('age_upon_outcome_in_weeks', 'N/A')}"),
                    html.P(f"Outcome: {dff.iloc[row_idx].get('outcome_type', 'N/A')}")
                ])
            ])
        ]
    )

# Run the app inline in Jupyter Notebook, with debug enabled
if __name__ == '__main__':
    app.run_server(mode='inline', debug=True)
